In [0]:
import torch
from torch import nn
import torch.nn.functional as F
from functools import partial

In [0]:
# Utility Functions and Classes
def conv3(c_in, c_out, stride = 1): 
  return nn.Conv2d(c_in, c_out, 3, stride, padding=1, bias=False)

def conv1(c_in, c_out, stride = 1): 
  return nn.Conv2d(c_in, c_out, 1, stride, bias=False) 

class PrintBlock(nn.Module):
  def __init__(self, message):
    super().__init__()
    self.message = message
  def forward(self, x):
    print(self.message)
    print("input shape:", x.shape)
    return x

identity = lambda x: x

In [0]:
class ResBlock(nn.Module):
  def __init__(self, c_in, c_mid, stride, res_path = None):
    super().__init__()

    c_out = c_mid * 4

    self.body = nn.Sequential(
        
        conv1(c_in, c_mid, stride), 
        nn.BatchNorm2d(c_mid),
        nn.ReLU(),

        conv3(c_mid, c_mid),
        nn.BatchNorm2d(c_out),
        nn.ReLU(),
        
        conv1(c_mid, c_out),
        nn.BatchNorm2d(c_out))
    
    self.res_path = identity if res_path is None else res_path

  def forward(self, inp): return F.relu(self.body(inp) + self.res_path(inp))

In [0]:
def resnet50(n_classes):

  start = nn.Sequential(
      nn.Conv2d(3, 64, 7, stride=2, bias = False, padding=3),
      nn.ReLU(),
      nn.BatchNorm2d(64),
      nn.MaxPool2d(kernel_size = 3, stride = 2)
  )

  c_in = 64

  model = []
  layer_sizes = [3, 4, 6, 3]
  #layer_sizes = [1,1,1,1]

  # Corresponds to layers conv{3,4,5}_1 in paper.
  downsample_layers = {1,2,3}
  for i, layer in enumerate(layer_sizes):
    for j in range(layer):
      print(i, j, "!")
      c_mid = c_in if i == 0 and j == 0 else (c_in // 2 if j == 0 else c_in // 4)
      stride = 2 if i in downsample_layers and j == 0 else 1
      
      res_path = nn.Sequential(
          conv1(c_in, c_mid * 4, stride),
          nn.BatchNorm2d(c_mid * 4)
      )

      model.append(PrintBlock(f"starting layer {i}-{j}"))
      model.append(ResBlock(c_in, c_mid, stride, res_path = res_path))
      c_in = c_mid * 4

  head = nn.Sequential(
      nn.AdaptiveAvgPool2d((1,1)),
      nn.Flatten(1),
      nn.Linear(2048, n_classes)
    )

  return nn.Sequential(start, *model, head)
  

In [0]:
m = resnet50(10).cuda()

0 0 !
0 1 !
0 2 !
1 0 !
1 1 !
1 2 !
1 3 !
2 0 !
2 1 !
2 2 !
2 3 !
2 4 !
2 5 !
3 0 !
3 1 !
3 2 !


In [0]:
m

Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): ReLU()
    (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (1): PrintBlock()
  (2): ResBlock(
    (body): Sequential(
      (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
      (6): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (7): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (res_path): Sequential(
      (0): Conv2d(64, 256, kernel_size=(1, 1), st

In [0]:
m(torch.randn(3, 3 ,400 ,400).cuda())

starting layer 0-0
input shape: torch.Size([3, 64, 99, 99])
starting layer 0-1
input shape: torch.Size([3, 256, 99, 99])
starting layer 0-2
input shape: torch.Size([3, 256, 99, 99])
starting layer 1-0
input shape: torch.Size([3, 256, 99, 99])
starting layer 1-1
input shape: torch.Size([3, 512, 50, 50])
starting layer 1-2
input shape: torch.Size([3, 512, 50, 50])
starting layer 1-3
input shape: torch.Size([3, 512, 50, 50])
starting layer 2-0
input shape: torch.Size([3, 512, 50, 50])
starting layer 2-1
input shape: torch.Size([3, 1024, 25, 25])
starting layer 2-2
input shape: torch.Size([3, 1024, 25, 25])
starting layer 2-3
input shape: torch.Size([3, 1024, 25, 25])
starting layer 2-4
input shape: torch.Size([3, 1024, 25, 25])
starting layer 2-5
input shape: torch.Size([3, 1024, 25, 25])
starting layer 3-0
input shape: torch.Size([3, 1024, 25, 25])
starting layer 3-1
input shape: torch.Size([3, 2048, 13, 13])
starting layer 3-2
input shape: torch.Size([3, 2048, 13, 13])


tensor([[-0.1001, -0.3788, -0.0488, -0.1361, -0.0988,  0.0832,  0.0424,  0.3383,
         -0.1861,  0.3094],
        [-0.0993, -0.3623,  0.0455, -0.1304, -0.1285,  0.1870,  0.0832,  0.3666,
         -0.1883,  0.4821],
        [ 0.0014, -0.3659, -0.0121, -0.1141, -0.0814,  0.1137, -0.0075,  0.3461,
         -0.1726,  0.3194]], device='cuda:0', grad_fn=<AddmmBackward>)

In [0]:
x(torch.randn(3,3,400,400))

tensor([[-1.2276, -0.4200,  0.5106,  ...,  0.2526, -0.1850, -0.7503],
        [-1.0671, -0.3666,  0.4834,  ...,  0.2535, -0.1636, -0.7538],
        [-1.1627, -0.4018,  0.5617,  ...,  0.2676, -0.2060, -0.8499]],
       grad_fn=<AddmmBackward>)